In [ ]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [ ]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

In [ ]:
S_N_Sequence_fen, S_N_Sequence_fen_short = load_s_n_fens("S_n_seq_rook")

> Potentielles TODO: Noch die anderen Tabellen von Python-Chess als Referenz / Vergleich aufführen (Gaviota)
> TODO: Beschreibung der Funktion
> TODO: Überlegung, ob Funktion noch notwendig ist

In [ ]:
def fen_in_lower_sequence(fen, sequence_index, s_n_short):
    for s in s_n_short[:sequence_index]:
        if fen in s:
            return True
    return False

> TODO: Beschreibung der Funktion
> TODO: Funktion wieder in weiß und schwarz teilen, möglicherweise hierzu nochmal loop teilen

In [ ]:
def every_move_of_sequence_in_lower(sequence_index, s_n_fen, s_n_short):
    print("Checking S_" + str(sequence_index) + "...")
    sequence_fen = s_n_fen[sequence_index]
    for fen in sequence_fen:
        cur_board = chess.Board(fen)
        if not cur_board.turn:
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_short = get_board_and_turn(cur_board.fen())
                if not fen_in_lower_sequence(cur_short, sequence_index, s_n_short):
                    print("Fen: " + cur_short + " not in lower S")
                    return False
                cur_board.pop()
            return True
        else:
            in_lower = False
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_short = get_board_and_turn(cur_board.fen())
                if fen_in_lower_sequence(cur_short, sequence_index, s_n_short):
                    in_lower = True
                cur_board.pop()
            return in_lower


In [ ]:
for i in range(len(S_N_Sequence_fen)):
    every_move_of_sequence_in_lower(i, S_N_Sequence_fen, S_N_Sequence_fen_short)

> TODO: Beschreibung der Funktion
> TODO: Fragwürdig wie sinnig die Fkt. ist, wenn sie in 99% der Fälle nicht zum Ende kommt, Überlegung noch anderen Test zu berücksichtigen
> TODO: Alternative finden (entweder Gaviota oder doch eine Methode, wie man abschätzen kann, ob es falsch zugeordnet worden ist)

In [ ]:
def find_move_count(sequence_index, fen, move_count, move_set, s_n_short):
    cur_board = chess.Board(fen)
    moves = cur_board.legal_moves
    if moves.count() == 0 and sequence_index == 0:
        move_set.add(move_count)
        return move_set
    if not cur_board.turn:
        # print("Black:")
        for move in cur_board.legal_moves:
            cur_board.push(move)
            move_count += 1
            cur_fen = cur_board.fen()
            new_seq = find_situation_in_sequence(cur_fen, s_n_short)
            # print(new_seq, ";" , cur_fen)
            move_set = find_move_count(new_seq, cur_fen, move_count, move_set, s_n_short)
            move_count -= 1
            cur_board.pop()
    else:
        # print("White:")
        for move in cur_board.legal_moves:
            cur_board.push(move)
            cur_fen = get_board_and_turn(cur_board.fen())
            if fen_in_lower_sequence(cur_fen, sequence_index, s_n_short):
                new_seq = find_situation_in_sequence(cur_fen, s_n_short)
                move_count += 1
                move_set = find_move_count(new_seq, cur_fen, move_count, move_set, s_n_short)
                move_count -= 1
            cur_board.pop()
    return move_set

In [ ]:
find_move_count(20, "8/8/8/3K4/8/5R2/2k5/8 b - - 0 1", 0, set(), S_N_Sequence_fen_short)

> TODO: Beschreibung der Funktion
> Überlegung: Auch sinnfrei, da sie für größere n nicht funktioniert -> ALTERNATIVE

In [ ]:
def check_boards_in_correct_sequence(sequence_index, s_n_fen, s_n_short):
    invalid = 0
    print("Comparing S_" + str(sequence_index) + "...")
    for fen in s_n_fen[sequence_index]:
        move_set = find_move_count(sequence_index, fen, 0, set(), s_n_short)
        if max(move_set) != sequence_index:
            invalid += 1
            print(fen)
    print("Invalid boards: " + str(invalid))

In [ ]:
check_boards_in_correct_sequence(4, S_N_Sequence_fen, S_N_Sequence_fen_short)

> TODO: Beschreibung der Funktion
> TODO: syzygy-Tabellen sollten ebenfalls zur Verfügung stellen, dass Stroetmann nicht downloaden muss

In [ ]:
def compare_with_syzygy(syzygy, s_n_sequence):
    counter = 0

    for n in range(len(s_n_sequence)):
        count = 0
        for fen in s_n_sequence[n]:
            chess_board = chess.Board(fen)
            if n != abs(syzygy.probe_dtz(chess_board)) != n + 1:
                count += 1
        print(f"S{n}: Syzygy believes {count} of {len(s_n_sequence[n])} Situations are wrongly placed in the sequence.")
    print(f"Syzygy believes {counter} Situations are wrongly placed in the sequence.")
    return count

In [ ]:
SYZYGY = chess.syzygy.Tablebase()
SYZYGY.add_directory("./syzygy")

In [ ]:
compare_with_syzygy(SYZYGY, S_N_Sequence_fen)